<a href="https://colab.research.google.com/github/ApplesDaddy/CMPT353Project/blob/rf_regress/regression_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!conda --version
!which python
!python --version
!echo $PYTHONPATH

In [ ]:
%env PYTHONPATH=
!echo $PYTHONPATH

In [ ]:
# most recent install link from https://docs.anaconda.com/free/miniconda/miniconda-other-installer-links/
!wget https://repo.anaconda.com/miniconda/Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!chmod +x Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!bash ./Miniconda3-py310_24.1.2-0-Linux-x86_64.sh -b -f -p /usr/local/

In [ ]:
!which conda
!conda --version
!conda install --channel defaults conda python=3.10 --yes
!conda update --channel defaults --all --yes

In [ ]:
import sys
sys.path

In [ ]:
!ls /usr/local/lib/python3.10/dist-packages

In [ ]:
sys.path.append("/usr/local/lib/python3.10/site-packages")
!conda config --add channels conda-forge

In [ ]:
!conda create --name musemotion_meta -c conda-forge numpy pandas matplotlib scikit-learn wandb pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

In [ ]:
# !conda init
# !conda activate musemotion_meta
# credit https://stackoverflow.com/a/62668276
%%shell
eval "$(conda shell.bash hook)"
conda activate musemotion_meta

In [1]:
pip install wandb -qU

# Load data
_Note_: tempo = bpm

In [2]:
!mkdir datasets
import urllib.request
urllib.request.urlretrieve("https://github.com/zhandrew4/music-mood-classification/raw/main/muse_v3.csv", "datasets/muse_v3.csv")
urllib.request.urlretrieve("https://github.com/zhandrew4/music-mood-classification/raw/main/full_data.csv", "datasets/full_data.csv")

mkdir: cannot create directory ‘datasets’: File exists


KeyboardInterrupt: 

In [1]:
import pandas as pd
sample = pd.read_csv("datasets/full_data.csv", nrows=3)
sample.columns

Index(['Unnamed: 0', 'track', 'artist', 'seeds', 'spotify_id', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature'],
      dtype='object')

In [2]:
KEEP_COLS_MASTER = ["track", "album", "artist", "release_date", "length",
             "tempo", "key", "time_signature",
             "mood"]

KEEP_COLS_MODEL = ["tempo", "key", "loudness", "time_signature", "duration_ms"]

# songs_df = pd.read_csv("data_moods.csv", usecols=KEEP_COLS_MASTER)
# make names the indices
songs_df = pd.read_csv("datasets/full_data.csv", usecols=["track", "artist"]+KEEP_COLS_MODEL, parse_dates=True, infer_datetime_format=True)
songs_df.set_index(["track", "artist"], inplace=True)
songs_df

/tmp/ipykernel_51915/2145830631.py:9: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  songs_df = pd.read_csv("datasets/full_data.csv", usecols=["track", "artist"]+KEEP_COLS_MODEL, parse_dates=True, infer_datetime_format=True)


,,key,loudness,tempo,duration_ms,time_signature
track,artist,,,,,
'Till I Collapse,Eminem,1,-3.237,171.447,297787,4
St. Anger,Metallica,2,-2.642,185.252,441133,4
Speedin',Rick Ross,1,-4.226,100.059,204960,4
Bamboo Banga,M.I.A.,9,-4.554,125.984,298360,4
Die MF Die,Dope,5,-3.524,126.020,186067,4
...,...,...,...,...,...,...
Secret,Quietdrive,0,-5.126,167.996,258373,4
The Last of the Rest Was the End,Medications,2,-5.774,144.844,324000,3
Lovechild,Daniel Lanois,10,-20.091,79.476,516280,4


Convert dates to numbers, specfically seconds.

_Note_: This goddamn dataset only gives the year for some songs.

In [3]:
# print(songs_df[songs_df["release_date"]=="1993"])
# # credit https://stackoverflow.com/a/54312941
# songs_df["release_date"] = pd.to_datetime(songs_df["release_date"], format="mixed").astype(int)/ 10**9
# songs_df["release_date"]

Load the V, A, D dimension values from the original MuSe dataset and add them to the rest of the data.
^ Join V, A, D columns by key columns title and author.

In [4]:
import pandas as pd

LABELS_COLS = ['valence_tags', 'arousal_tags', 'dominance_tags']

orig_dataset = pd.read_csv("datasets/muse_v3.csv")
orig_dataset = pd.read_csv("datasets/muse_v3.csv", usecols=["track", "artist"]+LABELS_COLS)
orig_dataset.set_index(["track", "artist"], inplace=True)
orig_dataset

,,valence_tags,arousal_tags,dominance_tags
track,artist,,,
'Till I Collapse,Eminem,4.550000,5.273125,5.690625
St. Anger,Metallica,3.710000,5.833000,5.427250
Speedin',Rick Ross,3.080000,5.870000,5.490000
Bamboo Banga,M.I.A.,6.555071,5.537214,5.691357
Die MF Die,Dope,3.771176,5.348235,5.441765
...,...,...,...,...
Battle,Gaby Hoffmann & Jay Duplass,5.370000,3.450000,5.330000
Hands Tied to the Roots of a Hemorrhage,Omar Rodriguez-Lopez,5.797887,4.132254,5.570563
Crookfield Zoo,Ace White,5.370000,3.450000,5.330000


In [5]:
VAL_COLS = ["valence_tags","arousal_tags","dominance_tags"]
songs_df = songs_df.join(orig_dataset, how="inner")

In [6]:
songs_df

,,key,loudness,tempo,duration_ms,time_signature,valence_tags,arousal_tags,dominance_tags
track,artist,,,,,,,,
'Till I Collapse,Eminem,1,-3.237,171.447,297787,4,4.550000,5.273125,5.690625
St. Anger,Metallica,2,-2.642,185.252,441133,4,3.710000,5.833000,5.427250
Speedin',Rick Ross,1,-4.226,100.059,204960,4,3.080000,5.870000,5.490000
Bamboo Banga,M.I.A.,9,-4.554,125.984,298360,4,6.555071,5.537214,5.691357
Die MF Die,Dope,5,-3.524,126.020,186067,4,3.771176,5.348235,5.441765
...,...,...,...,...,...,...,...,...,...
Secret,Quietdrive,0,-5.126,167.996,258373,4,5.370000,3.450000,5.330000
The Last of the Rest Was the End,Medications,2,-5.774,144.844,324000,3,5.370000,3.450000,5.330000
Lovechild,Daniel Lanois,10,-20.091,79.476,516280,4,6.685000,4.405000,5.625000


In [7]:
# Check for duplicates
songs_df.index.is_unique

True

Model will take data as (`X_train`,`y_train`)
- `X_train` is a 2D NumPy array of shape `n_samples` x `n_features`, i.e. one song per row, one metadata feature per column
- `y_train` is just convert the "mood" column into a 1D NumPy array

In [8]:
# How much should we take for training, testing?
songs_df.index.size

0.8 * songs_df.index.size

49301.600000000006

In [49]:
import numpy as np
from sklearn.model_selection import train_test_split

DROP_COLS = VAL_COLS#+["artist"]
# use a constant random state for reproducibility
# X = songs_df.drop(columns=DROP_COLS)
# y = songs_df[VAL_COLS]
X_train, X_test, y_train, y_test = train_test_split(songs_df.drop(columns=DROP_COLS), songs_df[VAL_COLS],
                                                    test_size=0.2, random_state=42)

# Model

## Set up Weights & Biases

In [10]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: t0m_x. Use `wandb login --relogin` to force relogin


True

## Multi-Layer Perceptron
[PyTorch tutorial](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

[Building a Regression Model in PyTorch](https://machinelearningmastery.com/building-a-regression-model-in-pytorch/)

[ChatGPT PyTorch Regression Model](https://chat.openai.com/share/8e1237ca-a45f-4abc-8740-25ca00d72bbf)

In [11]:
# from sklearn.neural_network import MLPRegressor

# mlp = MLPRegressor(solver='lbfgs', hidden_layer_sizes=())

# # terrible score lol
# mlp_regression = mlp.fit(X_train, y_test)
# print(mlp_regression.score(X_test, y_test))

In [12]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using {device} device")

Using cuda device


In [14]:
print(type(X_train), X_train)

<class 'pandas.core.frame.DataFrame'>                                                                     key  \
track                                       artist                        
Newst Flurries                              Brad Barr                11   
Detting Te Deum                             Georg Friedrich Händel    2   
Look What You've Done                       Tree63                    3   
Alpha Incipient                             Fallujah                  6   
Actors                                      Still Parade              6   
...                                                                 ...   
People Of The World                         Chris de Burgh            5   
Electric                                    Davy Vain                 1   
Born Like This                              Three Days Grace          7   
Stay One                                    Tom Burbank               0   
Within You Without You/Tomorrow Never Knows The Beatles       

In [15]:
print(X_test)

                                              key  loudness    tempo  \
track                artist                                            
Les OS               The Unicorns               7    -7.705  171.996   
The Millennium       The Flood Plain           11    -7.086   88.444   
Spring               The Innocence Mission     11   -11.309  141.594   
Heavenly Day         Patty Griffin             11    -7.385  184.241   
Finger Twist & Split Cute Is What We Aim For    9    -4.698  174.279   
...                                           ...       ...      ...   
In A Needle          The Used                   2    -4.565  119.978   
Gnome Dosed          Androcell                  4    -6.931  185.947   
My Apology           Floetry                   10    -7.808   98.680   
Preamble             Kalliope                   4    -8.669   82.461   
All Cats Are Grey    The Cure                   9   -13.913  110.114   

                                              duration_ms  time

In [16]:
print(y_test)

                                              valence_tags  arousal_tags  \
track                artist                                                
Les OS               The Unicorns                 5.634675      4.371169   
The Millennium       The Flood Plain              2.393333      2.526667   
Spring               The Innocence Mission        6.905000      2.925000   
Heavenly Day         Patty Griffin                6.561579      3.868421   
Finger Twist & Split Cute Is What We Aim For      7.637500      4.900000   
...                                                    ...           ...   
In A Needle          The Used                     6.840000      3.810000   
Gnome Dosed          Androcell                    5.581277      3.390426   
My Apology           Floetry                      6.677647      3.392206   
Preamble             Kalliope                     6.001905      4.455714   
All Cats Are Grey    The Cure                     4.162949      3.578590   

           

In [17]:
print(y_train)

                                                                    valence_tags  \
track                                       artist                                 
Newst Flurries                              Brad Barr                   7.212500   
Detting Te Deum                             Georg Friedrich Händel      6.950000   
Look What You've Done                       Tree63                      6.844667   
Alpha Incipient                             Fallujah                    5.753333   
Actors                                      Still Parade                2.860539   
...                                                                          ...   
People Of The World                         Chris de Burgh              7.105000   
Electric                                    Davy Vain                   3.850000   
Born Like This                              Three Days Grace            4.439500   
Stay One                                    Tom Burbank                 3.05

In [18]:
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_test: (12326, 5)
Shape of y_test: (12326, 3)


In [19]:
# credit: https://stackoverflow.com/questions/71198218/the-simple-mlp-nn-for-regression-in-pytorch-very-slow-learning-rev2

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_p=0.5):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1).double()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2).double()
        self.fc3 = nn.Linear(hidden_size2, output_size).double()
        self.dropout = nn.Dropout(dropout_p).double()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [20]:
from torch.utils.data import DataLoader, TensorDataset
# # convert dataframe to PyTorch Tensor
# X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
# X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
# y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
# y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 64

Clean up data retrieving and storage with a [custom PyTorch DataSet class](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

In [50]:
from torch.utils.data import DataLoader, TensorDataset
batch_size = 64

class SongMetaDataSet(torch.utils.data.Dataset):
    def __init__(self, X: pd.DataFrame, y: pd.DataFrame):
        """
        X is a tensor or DataFrame of the input features (i.e. song metadata), one row per song.
        y is a tensor or DataFrame of the target values (i.e. valence, arousal, dominance), one row per song.
        __All must have the (song name, artist) as the multi-index.__
        """
        # sort all by index to ensure they match
        for inp in (X, y):
            if not isinstance(inp, pd.DataFrame):
                raise ValueError(f"{inp} must be a pandas DataFrame")
            if len(inp.index.names) < 2:
                raise ValueError(f"{inp} must have song names and artists in its index")
        if len(X.index) != len(y.index):
            raise ValueError("X and y must be the same length")

        self.X = X
        # self.X.sort_index(inplace=True)
        self.y = y
        # self.y.sort_index(inplace=True)

    def __len__(self):
        return len(self.X.index)

    def __getitem__(self, idx) -> tuple[tuple[str,str], torch.Tensor, torch.Tensor]:
        """
        The returned tuple contains:
            1. a tuple of the song's (name, artist)
            2. the input features vector for this song
            3. the target vector (V, A, D values) for this song
        """
        # keep intermediate for y to extract index = (name, artist)
        labels_df = self.y.iloc[[idx]]
        if len(labels_df.index) > 1:
            print("Warning: indexing DataSet gave more than one song, only keeping the first")
        # > PyTorch's DataLoader will automatically stack the arrays along the first dimension (axis 0) when creating a batch
        #       ^ credit: Google Colab AI
        # but don't care, don't need to batch during testing
        IDs_pair = labels_df.index.values[0]
        return IDs_pair, torch.from_numpy(self.X.values[idx]), torch.from_numpy(labels_df.values)

def make_song_dataset(songs_df: pd.DataFrame, input_cols: list[str], label_cols: list[str]) -> SongMetaDataSet:
    """
    songs_df is a DataFrame of song metadata, with the song names as the index.
    input_cols is a list of columns to use as the input features (i.e. tempo, key, loudness).
    id_cols is a list of columns to include in the identifying information for each song.
    label_cols is a list of columns to use as the target values (i.e. valence, arousal, dominance).
    """
    X = songs_df[input_cols]
    y = songs_df[label_cols]
    return SongMetaDataSet(X, y)

In [51]:
# songs_df.loc[X_test.index]

In [52]:
# train_idx = X_train.index.values
# test_idx = X_test.index.values
# print(songs_df.loc[train_idx, KEEP_COLS_MODEL], songs_df.loc[train_idx, VAL_COLS], songs_df.loc[train_idx, "artist"])

train_dataset = make_song_dataset(songs_df.loc[X_train.index], KEEP_COLS_MODEL, VAL_COLS)
test_dataset = make_song_dataset(songs_df.loc[X_test.index], KEEP_COLS_MODEL, VAL_COLS)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True)

In [53]:
train_dataset[0]

(('Newst Flurries', 'Brad Barr'),
 tensor([ 7.9744e+01,  1.1000e+01, -1.3380e+01,  4.0000e+00,  2.6723e+05],
        dtype=torch.float64),
 tensor([[7.2125, 4.1975, 6.4750]], dtype=torch.float64))

In [54]:
for loader in [train_loader, test_loader]:
    print("Loader ", loader)
    for name_artist_pair, inputs, targets in loader:
        name, artist = name_artist_pair[0][0], name_artist_pair[1][0]
        print(f"{name} by {artist}")
        # print(f"{name_artist_pair[0]} by {name_artist_pair[1]}:")
        print("\tInputs:", inputs.shape, inputs.dtype)
        print("\tLabel:", targets.shape, inputs.dtype)
        break

Loader  <torch.utils.data.dataloader.DataLoader object at 0x7fa3a1dbb650>
The Plain of Ida by Blut aus Nord
	Inputs: torch.Size([64, 5]) torch.float64
	Label: torch.Size([64, 1, 3]) torch.float64
Loader  <torch.utils.data.dataloader.DataLoader object at 0x7fa3a1cd4b30>
Les OS by The Unicorns
	Inputs: torch.Size([1, 5]) torch.float64
	Label: torch.Size([1, 1, 3]) torch.float64


In [ ]:
# input_size = X_train.shape[1]
# hidden_size1 = 6
# hidden_size2 = 4
# output_size = y_train.shape[1]  # Assuming y_train has multiple output columns
# model = NeuralNetwork(input_size, hidden_size1, hidden_size2, output_size).to(device)

sweep_config = {
    "name": "MLP Sweep",
    "method": "grid",
    "metric": {"goal": "minimize", "name": "test loss"},
    "parameters": {
        "input_size": {"values": [X_train.shape[1]]},
        "hidden_size1": {"values": list(range(3,9))},
        "hidden_size2": {"values": list(range(3,9))},
        "output_size": {"values": [y_train.shape[1]]},
        "epochs": {"values": [5, 10, 15]},
        "dropout_p": {"values": [0.3, 0.5, 0.7]},
        "learning_rate": {"values": [0.001, 0.01, 0.1]}
    }
}
X_train.shape[1]

5

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Music Mood Prediction")

Create sweep with ID: lohmygvl
Sweep URL: https://wandb.ai/t0m_x/Music%20Mood%20Prediction/sweeps/lohmygvl


In [ ]:
import torch.optim as optim
import torch.utils
import torch.utils.data

TEST_TABLE_COLS = [
    "name",
    "artist",
    "tempo",
    "key signature",
    "loudness",
    "time signature",
    "duration (ms)",
    "predicted valence",
    "predicted arousal",
    "predicted dominance"
]

def train_epoch(train_loader, model, optimizer, criterion):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for _, inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

def test(test_loader: torch.utils.data.DataLoader, model: torch.nn.Module, criterion: torch.nn.Module):
    """record test examples and predictions in W&B table"""
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0

    test_table = wandb.Table(columns=TEST_TABLE_COLS)

    with torch.no_grad():
        for name_artist_pairs, inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)

            # Display and log some test examples <- input and output
            name, artist = name_artist_pairs[0][0], name_artist_pairs[1][0]
            print(f"{name} by {artist}")
            print(f'input: {inputs[0].cpu().numpy()}')
            print(f'output: {outputs[0].cpu().numpy()}')
            all_cells_vals = [name, artist] + inputs.tolist() + outputs.tolist()
            test_table.add_data(*all_cells_vals)

        test_loss /= len(test_loader.dataset)
        wandb.log({"test loss": test_loss})
        wandb.log({"prediction examples": test_table})
        print(f'test loss: {test_loss:.4f}')

        return test_loss

# Build and train a model
def pipeline(config=None, train_loader: torch.utils.data.DataLoader=train_loader) -> nn.Module:
    # Initialize a new wandb run for this hyperparameter configuration
    # config will be set by Sweep Controller
    with wandb.init(project="Music Mood Prediction", name="MLP Regressor", config=config):
        config = wandb.config

        # build the model
        model = NeuralNetwork(config["input_size"], config["hidden_size1"], config["hidden_size2"], config["output_size"], config["dropout_p"]).to(device)

        # define the loss function and the optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

        wandb.watch(model, criterion, log="all", log_freq=10)

        num_epochs = config["epochs"]
        avg_loss = 0.0
        for epoch in range(num_epochs):
            avg_loss = train_epoch(train_loader, model, optimizer, criterion)
            wandb.log({"epoch": epoch, "train loss": avg_loss})

        test_loss = test(test_loader, model, criterion)
        wandb.log({"test loss": test_loss})

        return model

wandb.agent(sweep_id, function=pipeline, count=6)

wandb: Agent Starting Run: rz2lkaow with config:
wandb: 	dropout_p: 0.3
wandb: 	epochs: 5
wandb: 	hidden_size1: 3
wandb: 	hidden_size2: 3
wandb: 	input_size: 5
wandb: 	learning_rate: 0.001
wandb: 	output_size: 3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64, 1, 3])) that is different to the input size (torch.Size([64, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([21, 1, 3])) that is different to the input size (torch.Size([21, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Traceback (most recent call last):
  File "/tmp/ipykernel_51915/2379712853.py", line 86, in pipeline
    test_loss = test(test_loader, model, criterion)
                ^^^^^^^^^^^^^^^^^^^^^^^^^

epoch,▁▃▅▆█
train loss,█▁▁▁▁
epoch,4
train loss,25.95429


Run rz2lkaow errored:
Traceback (most recent call last):
  File "/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_51915/2379712853.py", line 86, in pipeline
    test_loss = test(test_loader, model, criterion)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_51915/2379712853.py", line 45, in test
    outputs = model(inputs)
              ^^^^^^^^^^^^^
  File "/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1561, in _call_impl
    result = forward_call(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^

/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64, 1, 3])) that is different to the input size (torch.Size([64, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([21, 1, 3])) that is different to the input size (torch.Size([21, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Traceback (most recent call last):
  File "/tmp/ipykernel_51915/2379712853.py", line 86, in pipeline
    test_loss = test(test_loader, model, criterion)
                ^^^^^^^^^^^^^^^^^^^^^^^^^

...

epoch,▁▃▅▆█
train loss,█▁▁▁▁
epoch,4
train loss,14.45907


Run daxmikmc errored:
Traceback (most recent call last):
  File "/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_51915/2379712853.py", line 86, in pipeline
    test_loss = test(test_loader, model, criterion)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_51915/2379712853.py", line 45, in test
    outputs = model(inputs)
              ^^^^^^^^^^^^^
  File "/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1561, in _call_impl
    result = forward_call(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^

/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64, 1, 3])) that is different to the input size (torch.Size([64, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([21, 1, 3])) that is different to the input size (torch.Size([21, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Traceback (most recent call last):
  File "/tmp/ipykernel_51915/2379712853.py", line 86, in pipeline
    test_loss = test(test_loader, model, criterion)
                ^^^^^^^^^^^^^^^^^^^^^^^^^

epoch,▁▃▅▆█
train loss,█▁▁▁▁
epoch,4
train loss,2.25993


Run x83y36ea errored:
Traceback (most recent call last):
  File "/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_51915/2379712853.py", line 86, in pipeline
    test_loss = test(test_loader, model, criterion)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_51915/2379712853.py", line 45, in test
    outputs = model(inputs)
              ^^^^^^^^^^^^^
  File "/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1561, in _call_impl
    result = forward_call(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
# sweep_id = wandb.sweep(sweep_config, project="Music Mood Prediction")
# wandb.agent(sweep_id, function=pipeline)

In [ ]:
wandb.finish()

## Histogram-Based Gradient Boosted Tree
- [scikit-learn tutorial](https://scikit-learn.org/stable/modules/ensemble.html#histogram-based-gradient-boosting)
- [regressor API reference](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#sklearn.ensemble.HistGradientBoostingRegressor)

In [ ]:
wandb.init(
    project="Music Mood Prediction",
    name="Histogram-based Gradient Boosting Regressor"
    )

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_validate

hgbt = MultiOutputRegressor(HistGradientBoostingRegressor(random_state=0))
regression = hgbt.fit(X_train, y_train)

In [ ]:
r2 = regression.score(X_test, y_test)
wandb.log({"Coefficient of Correlation (R^2)": r2})
# The outliers chart only supports univariate (i.e. 1D output) regression
# wandb.sklearn.plot_regressor(regression,
#                             X_train, X_test,
#                             y_train, y_test,
#                             model_name="hist_gradient_boosting_regressor")
wandb.sklearn.plot_learning_curve(regression, X_train, y_train)
wandb.sklearn.plot_summary_metrics(regression, X_train, y_train, X_test, y_test)

/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold

param_grid = {
    "estimator__learning_rate": [0.1, 0.01, 0.001],
    # "estimator__n_estimators": [100, 200, 300],
    "estimator__max_depth": [3, 5, 7],
    # "estimator__min_samples_split": [20, 35, 50],
    "estimator__min_samples_leaf": list(range(1,6))
}

cv_fold = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(hgbt, param_grid, cv=cv_fold, n_jobs=-1, scoring="neg_mean_squared_error")
param_search_results = grid_search.fit(X_train, y_train)

best_model = param_search_results.best_estimator_
best_params = param_search_results.best_params_
wandb.log({"Coefficient of Correlation (R^2)": best_model.score(X_test, y_test)})

In [ ]:
wandb.sklearn.plot_learning_curve(best_model, X_train, y_train)
wandb.sklearn.plot_summary_metrics(best_model, X_train, y_train, X_test, y_test)

/usr/shared/CMPT/scratch/wxa38/miniforge3/envs/musemotion_meta/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [ ]:
wandb.finish()

Coefficient of Correlation (R^2),▁█
Coefficient of Correlation (R^2),0.04237
